# Задача: 
Для произвольно выбранного датасета провести обработку данных и построить предсказательную модель с использованием функционала pySpark.

In [24]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [40]:

# Инициализация SparkSession
spark = SparkSession.builder \
    .appName("Wine Quality Prediction") \
    .getOrCreate()

In [41]:
# Загрузка данных
df = spark.read.csv("./winequality-white.csv", sep=';', header=True, inferSchema=True)

In [42]:
df.show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.0|            0.27|       0.36|          20.7|    0.045|               45.0|               170.0|  1.001| 3.0|     0.45|    8.8|      6|
|          6.3|             0.3|       0.34|           1.6|    0.049|               14.0|               132.0|  0.994| 3.3|     0.49|    9.5|      6|
|          8.1|            0.28|        0.4|           6.9|     0.05|               30.0|                97.0| 0.9951|3.26|     0.44|   10.1|      6|
|          7.2|            0.23|       0.32|           8.5|    0.058|               47.0|           

In [43]:
# Удаление пропусков
df = df.na.drop()

In [45]:
# Определяем векторизатор
assembler = VectorAssembler(inputCols=df.columns[:-1], outputCol="features")
output = assembler.transform(df)

In [47]:
# Разделение на признаки и целевую переменную
X = assembler.transform(df).select("features")
y = df.select("quality")

In [13]:
# Масштабирование данных
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)
scalerModel = scaler.fit(X)
X = scalerModel.transform(X)

In [49]:
# Объединение признаков и целевой переменной
data = X.join(y, how="inner")

In [50]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test = data.randomSplit([0.8, 0.2], seed=42)

In [26]:
# Обучаем модель
lr = LogisticRegression(featuresCol="scaledFeatures", labelCol="quality")


In [27]:
# Обучение модели
model = lr.fit(X_train)

In [28]:
# Предсказание на тестовой выборке
y_pred = model.transform(X_test)

In [33]:
# Оценка качества модели
evaluator = MulticlassClassificationEvaluator(labelCol="quality", predictionCol="prediction")

accuracy = evaluator.evaluate(y_pred, {evaluator.metricName: "accuracy"})
f1 = evaluator.evaluate(y_pred, {evaluator.metricName: "f1"})

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.44877149218743645
F1 Score: 0.27802293637909675


In [51]:

# Остановка SparkSession
spark.stop()

# Вывод:
pySpark является заменой стандартных библиотек (pandas) для датафреймов больших размеров. 
Явным минусом pySpark является скорость(обучение модели заняло почти 5 минут в то время, как в pandas это занимает пару сеунд)